# **Suryakanta Karan (M22AIE207) m22aie207@iitj.ac.in**

In [ ]:
import os
from google.colab import drive
drive.mount('/content/drive', force_remount=True)
os.chdir('/content/drive/MyDrive/surya/DL_Assignment/Fractal-1_Assignment-1')

In [1]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import Input, Flatten, Dense
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.applications import VGG16
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, roc_auc_score, roc_curve, auc
import matplotlib.pyplot as plt

# Load and preprocess the STL-10 dataset (You should have this prepared)
# You can use your own data loading and preprocessing code here

# Load pre-trained autoencoder's encoder (Replace with your encoder model)
# For example, let's create a simple encoder with VGG16 as a placeholder
encoder_input = Input(shape=(96, 96, 3))
encoder_base = VGG16(input_tensor=encoder_input, include_top=False, weights='imagenet')
encoder_output = Flatten()(encoder_base.layers[-1].output)
encoder_model = Model(encoder_input, encoder_output)

# Build the downstream task classifier (MLP)
def build_classifier(input_shape, hidden_layers):
    input_layer = Input(shape=input_shape)
    x = input_layer
    for units in hidden_layers:
        x = Dense(units, activation='relu')(x)
    output_layer = Dense(10, activation='softmax')(x)  # Assuming 10 classes
    classifier = Model(input_layer, output_layer)
    return classifier

# Choose hidden layer configurations
hidden_layers_3 = [256, 128, 64]  # Task 2.3.a
hidden_layers_5 = [512, 256, 128, 64, 32]  # Task 2.3.b

# Initialize variables to store results
results = {}

# Fine-tune the classifier with different percentages of training samples
percentages = [1, 10, 20, 40, 60]  # Task 2.4
for percent in percentages:
    # Split the dataset based on the percentage
    train_x, _, train_y, _ = train_test_split(dataset_x, dataset_y, train_size=percent / 100, stratify=dataset_y)

    # Build the classifier
    hidden_layers = hidden_layers_3 if percent <= 10 else hidden_layers_5
    classifier = build_classifier(encoder_model.output.shape[1:], hidden_layers)

    # Compile the classifier
    classifier.compile(optimizer=Adam(learning_rate=0.001), loss='categorical_crossentropy', metrics=['accuracy'])

    # Fine-tune the classifier on the STL-10 dataset
    classifier.fit(train_x, train_y, batch_size=32, epochs=10, validation_split=0.2)

    # Evaluate the classifier and store the results
    test_x, test_y = load_test_data()  # Load your test data
    accuracy = classifier.evaluate(test_x, test_y)[1]
    results[percent] = accuracy

# Prepare confusion matrix and AUC-ROC curve for selected configurations
selected_configurations = [hidden_layers_3, hidden_layers_5]  # Task 2.3.a and 2.3.b
for config in selected_configurations:
    classifier = build_classifier(encoder_model.output.shape[1:], config)
    classifier.compile(optimizer=Adam(learning_rate=0.001), loss='categorical_crossentropy', metrics=['accuracy'])
    classifier.fit(train_x, train_y, batch_size=32, epochs=10, validation_split=0.2)

    # Evaluate the classifier
    test_x, test_y = load_test_data()  # Load your test data
    predictions = classifier.predict(test_x)
    confusion = confusion_matrix(np.argmax(test_y, axis=1), np.argmax(predictions, axis=1))

    # Calculate AUC-ROC
    fpr, tpr, thresholds = roc_curve(test_y, predictions)
    roc_auc = auc(fpr, tpr)

    # Store the results for this configuration
    results[config] = {'confusion_matrix': confusion, 'roc_auc': roc_auc}

# Implement a different architecture (Task 2.6)
# You can try different architectures and fine-tuning strategies here

# Print or visualize the results as needed
print(results)


58889256/58889256 [==============================] - 0s 0us/step


NameError: ignored